In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/MyDrive
!mount --bind /content/drive/My\ Drive /content/MyDrive
# the magic % will help with the cd command :)
%cd /content

In [ ]:
!pip install scanpy
!pip install squidpy
!pip install scikit-image
!pip install tangram-sc
!pip install gdown
# to upgrade
!pip install --upgrade gdown

In [2]:
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd
from anndata import AnnData
import pathlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
import seaborn as sns
import tangram as tg

In [2]:
%cd '/content/MyDrive/spatial_scRNAseq_test'

[WinError 3] The system cannot find the path specified: "'/content/MyDrive/spatial_scRNAseq_test'"
C:\Users\juhyu\Documents\GitHub\Data_Science_Capstone_Fall_2022


In [3]:
#!gdown --fuzzy 'https://drive.google.com/file/d/19O5HyRPUcaeYZut7m_wC7ND6Tb_vNVle/view?usp=share_link'

# 1. Load in Files

In [3]:
adata_sc = sc.read_h5ad("pasca.log1p_liger_med_singleR_noglyc.h5ad")

In [4]:
adata_st = sq.datasets.visium('V1_Human_Brain_Section_1')   
#adata_st = sq.datasets.visium('V1_Human_Brain_Section_2') 

In [5]:
img = sq.datasets.visium_fluo_image_crop()
#img = sq.datasets.visium_hne_image_crop()

# 2. Preprocess spatial data

In [6]:
sc.pp.calculate_qc_metrics(adata_st, inplace=True)

In [7]:
adata_st

AnnData object with n_obs × n_vars = 4910 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'spatial'
    obsm: 'spatial'

In [8]:
sc.pp.normalize_total(adata_st, inplace=True)
sc.pp.log1p(adata_st)
sc.pp.highly_variable_genes(adata_st, flavor="seurat", n_top_genes=2000, inplace=True)

In [9]:
sc.pp.neighbors(adata_st)
sc.tl.umap(adata_st)
sc.tl.leiden(adata_st, key_added="clusters")

         Falling back to preprocessing with `sc.pp.pca` and default params.


In [12]:
# Pre processing of adata_st ends

In [13]:
#adata_st.write_h5ad("adata_st_postpre.h5ad")

In [14]:
#adata_st = sc.read_h5ad("adata_st_postpre.h5ad")

# 3. Generate Map

## 3.1 Select Genes

In [10]:
adata_sc.uns['log1p']["base"] = None

### 3.1.1 Rank Gene Groups (Recommended)

In [11]:
sc.tl.rank_genes_groups(adata_sc, groupby="nowakowski_med", use_raw=False)
markers_df = pd.DataFrame(adata_sc.uns["rank_genes_groups"]["names"]).iloc[0:100, :]
markers = list(np.unique(markers_df.melt().value.values))
len(markers)

1320

### 3.1.2 Highly variable Genes (Alternative)

In [19]:
'''
sc.pp.highly_variable_genes(adata_sc)
highvar_df = pd.DataFrame(adata_sc.var["highly_variable"])
highvar_df[highvar_df["highly_variable"] == True]
highvar_markers = list(highvar_df[highvar_df["highly_variable"] == True].index.values)
print(highvar_markers[:20])
'''

'\nsc.pp.highly_variable_genes(adata_sc)\nhighvar_df = pd.DataFrame(adata_sc.var["highly_variable"])\nhighvar_df[highvar_df["highly_variable"] == True]\nhighvar_markers = list(highvar_df[highvar_df["highly_variable"] == True].index.values)\nprint(highvar_markers[:20])\n'

## 3.2 Single Anndata

### 3.2.1 Single Preprocess AnnData

In [1]:
tg.pp_adatas(adata_sc, adata_st, genes=markers)

NameError: name 'tg' is not defined

### 3.2.2 Single Generate Map

This is the most important process

In [22]:
import torch
torch.cuda.empty_cache()
ad_map = tg.map_cells_to_space(adata_sc, adata_st,
    mode="cells",
#     mode="clusters",
#     cluster_label='cell_subclass',  # .obs field w cell types
    density_prior='rna_count_based',
    num_epochs=10,
#     device="cuda:0",
    device='cpu',
)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 1301 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.495, KL reg: 0.010


INFO:root:Saving results..


In [23]:
#adata_st
#adata_st.write_h5ad("adata_st_postmap.h5ad")

In [ ]:
#adata_st = sc.read_h5ad("adata_st_postmap.h5ad")
#ad_map = sc.read_h5ad("ad_map.h5ad")

### 3.2.3 Save Map

In [26]:
adata_st.write_h5ad("adata_st.h5ad")
ad_map.write_h5ad("ad_map.h5ad")

## 3.3 Double Anndata

###  3.3.1 Separate to two single cell data

In [13]:
adata_sc.obs['sampleID']

AAACCTGAGCTCAACT-1-0    GSM4306931_Control_1
AAACCTGAGGAATGGA-1-0    GSM4306931_Control_1
AAACCTGAGTCGCCGT-1-0    GSM4306931_Control_1
AAACCTGAGTCTCCTC-1-0    GSM4306931_Control_1
AAACCTGAGTGCCATT-1-0    GSM4306931_Control_1
                                ...         
TTTGTCATCGCAAACT-1-3    GSM4306934_Patient_4
TTTGTCATCGTAGATC-1-3    GSM4306934_Patient_4
TTTGTCATCGTAGGAG-1-3    GSM4306934_Patient_4
TTTGTCATCTAACTTC-1-3    GSM4306934_Patient_4
TTTGTCATCTTGCCGT-1-3    GSM4306934_Patient_4
Name: sampleID, Length: 45214, dtype: category
Categories (4, object): ['GSM4306931_Control_1', 'GSM4306932_Control_2', 'GSM4306933_Patient_1', 'GSM4306934_Patient_4']

In [15]:
filter_val = ['GSM4306931_Control_1', 'GSM4306932_Control_2']
adata_sc1 = adata_sc[adata_sc.obs['sampleID'].isin(filter_val),:]
adata_sc2 = adata_sc[~adata_sc.obs['sampleID'].isin(filter_val),:]

### 3.3.2 Double Preprocess Data

In [16]:
tg.pp_adatas(adata_sc1, adata_st, genes=markers)
tg.pp_adatas(adata_sc2, adata_st, genes=markers)

INFO:root:1301 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:20358 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:1301 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:20823 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


### 3.3.3 Double Generate Map

In [17]:
epochs = 10 #500

import torch
torch.cuda.empty_cache()
ad_map1 = tg.map_cells_to_space(adata_sc1, adata_st,
    mode="cells",
#     mode="clusters",
#     cluster_label='cell_subclass',  # .obs field w cell types
    density_prior='rna_count_based',
    num_epochs= epochs,
#     device="cuda:0",
    device='cpu',
)
torch.cuda.empty_cache()
ad_map2 = tg.map_cells_to_space(adata_sc2, adata_st,
    mode="cells",
#     mode="clusters",
#     cluster_label='cell_subclass',  # .obs field w cell types
    density_prior='rna_count_based',
    num_epochs= epochs,
#     device="cuda:0",
    device='cpu',
)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 1301 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.494, KL reg: 0.010


INFO:root:Saving results..
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 1301 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.494, KL reg: 0.010


INFO:root:Saving results..


### 3.3.4 Double Save Map

In [18]:
adata_st

AnnData object with n_obs × n_vars = 4910 × 25285
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'clusters', 'uniform_density', 'rna_count_based_density'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'n_cells', 'sparsity'
    uns: 'spatial', 'log1p', 'hvg', 'neighbors', 'umap', 'leiden', 'training_genes', 'overlap_genes'
    obsm: 'spatial', 'X_pca', 'X_umap'
    obsp: 'distances', 'connectivities'

In [19]:
adata_st.write_h5ad("adata_st.h5ad")
ad_map1.write_h5ad("ad_map1.h5ad")
ad_map2.write_h5ad("ad_map2.h5ad")